In [1]:
import os
import random
import torch
import numpy as np
from nnunet.inference.predict import load_model_and_checkpoint_files
from sklearn.metrics import f1_score

os.system('jupyter nbconvert --to python Pytorch_model.ipynb')
from Pytorch_model import Encodeur_nnUnet, Multitask_nnUNet_diffuse_noSeg2

os.system('jupyter nbconvert --to python Pytorch_train_multitask_4diffuse_noSeg.ipynb')
from Pytorch_train_multitask_4diffuse_noSeg import train, evaluation

os.system('jupyter nbconvert --to python metric_calculations.ipynb')
from metric_calculations import *

seed = 0
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

/home/nguyen-k/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



[NbConvertApp] Converting notebook Pytorch_model.ipynb to python
[NbConvertApp] Writing 30093 bytes to Pytorch_model.py
[NbConvertApp] Converting notebook Pytorch_train_multitask_4diffuse_noSeg.ipynb to python
[NbConvertApp] Writing 15961 bytes to Pytorch_train_multitask_4diffuse_noSeg.py
[NbConvertApp] Converting notebook Pytorch_dataloader.ipynb to python
[NbConvertApp] Writing 24303 bytes to Pytorch_dataloader.py
[NbConvertApp] Converting notebook preprocessing_Pytorch.ipynb to python
[NbConvertApp] Writing 28401 bytes to preprocessing_Pytorch.py
[NbConvertApp] Converting notebook gradcam.ipynb to python
[NbConvertApp] Writing 6149 bytes to gradcam.py
[NbConvertApp] Converting notebook Pytorch_utils.ipynb to python
[NbConvertApp] Writing 98252 bytes to Pytorch_utils.py
[NbConvertApp] Converting notebook roc_Precision_Recall.ipynb to python
[NbConvertApp] Writing 4174 bytes to roc_Precision_Recall.py
[NbConvertApp] Converting notebook metric_calculations.ipynb to python
[NbConvertApp

In [2]:
#DATA
dim = (32*4, 32*3, 32*6)
spacing = 4

#IMAGE GENERATOR
scale = (0.80, 1.20)
sigma = (0.3, 0.8)
task = '001'
#001 : PET SUV - 002 : PET sans SUV - 003 - PET CT 2 Channels - 004 - PET CT Fusion

#MODELE
learning_rate = 1e-4
score_weight = 3
drop_encode = 0.5
l1_lambda_fc1, l2_lambda_fc1 = 1e-3, 1e-3
l1_lambda_fc1_df, l2_lambda_fc1_df = 8e-2, 8e-2
weight_decay = 3e-5

batch_size = 1
nb_epoch = 40
num_workers = 10
patience = 3

In [3]:
base = '/media/nguyen-k/nnUNet_trained_models/nnUNet/3d_fullres'
list_task = os.listdir(base)

for t in list_task :
    if task in t :
        folders = os.path.join(base, t, 'nnUNetTrainerV2__nnUNetPlansv2.1')

In [4]:
for ind_list in range(1) : 

    test = 'Multitask_PET_L'+str(ind_list)
    dir_base = '/home/nguyen-k/Bureau/segCassiopet2/Comparatif/Archives_4diffuse_noSeg/'+test
    try:
        os.mkdir(dir_base)
    except OSError as error: 
        print(error) 

    path_list = '/home/nguyen-k/Bureau/segCassiopet2/List_Patient_new_'+str(ind_list)
    list_test = list(np.load(path_list + '/Test/list_test.npy'))
    test_label_classe = np.load(path_list + '/Test/test_label_classe.npy')
    test_label_diffuse = np.load(path_list + '/Test/test_label_diffuse.npy') 
    test_label_dfo = np.load(path_list + '/Test/test_label_dfo.npy')   
    test_prob = np.zeros((len(list_test), 3))
    np.save(dir_base+'/list_test.npy', list_test)
    np.save(dir_base+'/test_label_classe.npy', test_label_classe)
    np.save(dir_base+'/test_label_diffuse.npy', test_label_diffuse)

    for fold in range(1, 6) :
        print('LIST PATIENT', ind_list, ' - FOLD', fold)  

        dir_p = dir_base+'/Fold'+str(fold)
        dir_p_1 = dir_p+'/Fig_seg_val'
        dir_p_2 = dir_p+'/Fig_seg_test'
        dir_p_3 = dir_p+'/attention_maps'
        
        try:
            os.mkdir(dir_p)
        except:
            print('Dir existes...')

        try:
            os.mkdir(dir_p_1)
        except:
            print('Dir existes...')

        try:
            os.mkdir(dir_p_2)
        except:
            print('Dir existes...')
        
        try:
            os.mkdir(dir_p_3)
        except:
            print('Dir existes...')
        #TRAIN     
        trainer, params_tr = load_model_and_checkpoint_files(folders, folds=None, mixed_precision=None, checkpoint_name="model_best")

        nn_Unet = Encodeur_nnUnet(trainer.network)
        state_dict = trainer.network.state_dict()
        keys = list(state_dict.keys())

        for e in keys:
            if 'conv_blocks_context' not in e :
                del state_dict[e]
                keys = state_dict.keys()

        nn_Unet.load_state_dict(state_dict)
        MultitaskNet = Multitask_nnUNet_diffuse_noSeg2(nn_Unet, drop_encode=drop_encode, n_classes=3).cuda()   
        
        train(fold, MultitaskNet, nb_epoch, score_weight, l1_lambda_fc1, l2_lambda_fc1, l1_lambda_fc1_df, l2_lambda_fc1_df,dim, spacing, scale, sigma, 
                    num_workers, drop_encode, batch_size, learning_rate, patience, weight_decay, dir_p, path_list, seed)

        #EVALUATION
        print('LIST PATIENT', ind_list, ' - FOLD', fold, ' - VALIDATION')
        path_train_val = os.path.join(path_list, 'Fold'+ str(fold))
        list_val = list(np.load(path_train_val+'/list_val.npy'))
        val_label_classe = np.load(path_train_val+'/val_label_classe.npy')
        val_label_classe_diffuse = np.load(path_train_val+'/val_label_classe_diffuse.npy')
        val_label_classe_dfo = np.load(path_train_val+'/val_label_classe_dfo.npy')
        evaluation(MultitaskNet, list_val, val_label_classe, val_label_classe_diffuse, val_label_classe_dfo, dim, spacing, num_workers, dir_p_1)

        #TEST
        print('LIST PATIENT', ind_list, ' - FOLD', fold, ' - TEST')
        test_prob, test_prob_df = evaluation(MultitaskNet, list_test, test_label_classe, test_label_diffuse, test_label_dfo, dim, spacing, num_workers, dir_p_2)
        np.save(os.path.join(dir_p, 'test_prob.npy'), test_prob)
        np.save(os.path.join(dir_p, 'test_prob_df.npy'), test_prob_df)
        print(' ')        

        del MultitaskNet

[Errno 17] File exists: '/home/nguyen-k/Bureau/segCassiopet2/Comparatif/Archives_4diffuse_noSeg/Multitask_PET_L0'
LIST PATIENT 0  - FOLD 1
Dir existes...
Dir existes...
Dir existes...
Dir existes...
folds is None so we will automatically look for output folders (not using 'all'!)
found the following folds:  ['/media/nguyen-k/nnUNet_trained_models/nnUNet/3d_fullres/Task001/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0']
using the following model files:  ['/media/nguyen-k/nnUNet_trained_models/nnUNet/3d_fullres/Task001/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0/model_best.model']
list_class  [14, 9, 2, 47, 19, 4, 3, 2, 1, 36, 5, 2]
173 176 191
135 133 143 129
LIST PATIENT 0  - FOLD 1  - VALIDATION
Classification
Micro F1 score  0.6111111111111112 DF Micro F1 score  0.4444444444444444 NDF Micro F1 score  0.16666666666666666
Macro F1 score  0.3782312925170068
Weighted F1 score  0.6102040816326532
Micro F1 score df  0.4722222222222222 DF Micro F1 score  0.4722222222222222 NDF Micro F1 score  0.0
Mac

In [5]:
os.system('jupyter nbconvert --to python roc_Precision_Recall.ipynb')
from roc_Precision_Recall import *
#from sklearn.metrics import confusion_matrix

for ind_list in range(1) : 

    print('LIST PATIENT', ind_list)  

    test = 'Multitask_PET_L'+str(ind_list)
    dir_base = '/home/nguyen-k/Bureau/segCassiopet2/Comparatif/Archives_4diffuse_noSeg/'+test

    fold = 1
    dir_p = dir_base+'/Fold'+str(fold)
    sum = np.load(os.path.join(dir_p, 'test_prob.npy'))
    sum_df = np.load(os.path.join(dir_p, 'test_prob_df.npy'))

    path_list = '/home/nguyen-k/Bureau/segCassiopet2/List_Patient_new_'+str(ind_list)
    list_test = list(np.load(path_list + '/Test/list_test.npy'))
    test_label_classe = np.load(path_list + '/Test/test_label_classe.npy')   
    test_label_classe = np.array(test_label_classe, dtype=np.uint8)
    test_label_diffuse = np.load(path_list + '/Test/test_label_diffuse.npy')   
    test_label_diffuse = np.array(test_label_diffuse, dtype=np.uint8)
    test_label_dfo = np.load(path_list + '/Test/test_label_dfo.npy')   
    test_label_dfo = np.array(test_label_dfo, dtype=np.uint8)

    for fold in range(2, 6) :
        dir_p = dir_base+'/Fold'+str(fold)
        test_prob = np.load(os.path.join(dir_p, 'test_prob.npy'))
        sum += test_prob
        test_prob_df = np.load(os.path.join(dir_p, 'test_prob_df.npy'))
        sum_df += test_prob_df

    test_prob = sum / 5
    test_prob_df = sum_df / 5

    pred = np.zeros(test_label_classe.shape[0])
    for i in range(test_label_classe.shape[0]) : 
        pred[i] = np.argmax(test_prob[i])

    mat_label = np.zeros((test_label_classe.shape[0],3))
    for i in range(test_label_classe.shape[0]) :
        mat_label[i, int(test_label_classe[i])] = 1
    
    micro_f1_DF, micro_f1_NDF, _ = get_accuracy(test_label_classe.tolist(),pred.tolist(),[0,1,2],test_label_dfo)
    

    roc_auc, fpr, tpr = compute_ROC_auc(y_label=mat_label,y_predicted=test_prob,n_classes=3)
    plt.clf()
    plot_ROC_curve(fpr,tpr,roc_auc,classe=0,color='blue')
    plot_ROC_curve(fpr,tpr,roc_auc,classe=1,color='red')
    plot_ROC_curve(fpr,tpr,roc_auc,classe=2,color='black')
    plt.savefig(dir_base+'/ROC.png')


    print('Micro F1 score ', f1_score(y_true=test_label_classe, y_pred=pred, average='micro'),
            'DF Micro F1 score ', micro_f1_DF, 'NDF Micro F1 score ', micro_f1_NDF)
    print('Macro F1 score ', f1_score(y_true=test_label_classe, y_pred=pred, average='macro'))
    print('Weighted F1 score ', f1_score(y_true=test_label_classe, y_pred=pred, average='weighted'))

    pred_df = np.zeros(test_label_diffuse.shape[0])
    for i in range(test_label_diffuse.shape[0]) : 
        pred_df[i] = np.argmax(test_prob_df[i])

    mat_label_df = np.zeros((test_label_diffuse.shape[0],4))
    for i in range(test_label_diffuse.shape[0]) :
        mat_label_df[i, int(test_label_diffuse[i])] = 1

    micro_f1_DF_df, micro_f1_NDF_df, _ = get_accuracy(test_label_diffuse,pred_df,[0,1,2,3],test_label_diffuse)
    roc_auc_df, fpr_df, tpr_df = compute_ROC_auc(y_label=mat_label_df,y_predicted=test_prob_df, n_classes=4)
    
    plt.clf()
    plot_ROC_curve(fpr_df,tpr_df,roc_auc_df,classe=0,color='blue')
    plot_ROC_curve(fpr_df,tpr_df,roc_auc_df,classe=1,color='red')
    plot_ROC_curve(fpr_df,tpr_df,roc_auc_df,classe=2,color='black')
    plot_ROC_curve(fpr_df,tpr_df,roc_auc_df,classe=3,color='green')
    plt.savefig(dir_base+'/ROC_df.png')

    print('Micro F1 score ', f1_score(y_true=test_label_diffuse, y_pred=pred_df, average='micro'),
            'DF Micro F1 score ', micro_f1_DF_df, 'NDF Micro F1 score ', micro_f1_NDF_df)
    print('Macro F1 score ', f1_score(y_true=test_label_diffuse, y_pred=pred_df, average='macro'))
    print('Weighted F1 score ', f1_score(y_true=test_label_diffuse, y_pred=pred_df, average='weighted'))

[NbConvertApp] Converting notebook roc_Precision_Recall.ipynb to python
[NbConvertApp] Writing 4174 bytes to roc_Precision_Recall.py


LIST PATIENT 0
Micro F1 score  0.5918367346938775 DF Micro F1 score  0.4897959183673469 NDF Micro F1 score  0.10204081632653061
Macro F1 score  0.28904109589041094
Weighted F1 score  0.5703941850712888
Micro F1 score  0.46938775510204084 DF Micro F1 score  0.3877551020408163 NDF Micro F1 score  0.08163265306122448
Macro F1 score  0.28174603174603174
Weighted F1 score  0.4026563006154843


In [6]:
roc_auc

{0: 0.6795918367346938, 1: 0.5813397129186603, 2: 0.818840579710145}

In [7]:
roc_auc_df

{0: 0.7111111111111111,
 1: 0.5050167224080268,
 2: 0.2021276595744681,
 3: 0.5098039215686274}